In [ ]:
# Unzips the dataset file - produces one rar file for each labeled action
# Caution: Should be run once
!unrar x hmdb51_org.rar extracts/

In [ ]:
# Unzips the produced rar files for each labeled action
# Caution: Should be run once
!for rarFile in extracts/*.rar; do unrar x $rarFile all_videos/; done

In [1]:
import csv
import pandas as pd
import os
import random
import sys
import json
import time

In [ ]:
def convert_avi_to_mp4(avi_file_path):
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = avi_file_path[0:len(avi_file_path)-4]))
    return True

In [ ]:
# Conver all videos from avi to mp4
# Caution: Should be run once
path = "all_videos"

for root, d_names, f_names in os.walk(path):
    for fileName in f_names:
        convert_avi_to_mp4(f"{root}/{fileName}")
        time.sleep(.02)

In [10]:
# delete all avi files
# WARNING: make sure there is no important avi files in the working directory
!find . -name "*.avi" -type f -delete

In [2]:
# Creates the annotations following the exisiting annotation for ssv2
# Caution: Should be run once

# Name of the dir that holds the videos
path = "hmdb51"
next_free_slot = 174 # ssv2 has labels upto 173

label_maker = dict()
test_set = [ ]
val_set = [ ]
train_set = [ ]

for root, d_names, f_names in os.walk(path):
    for fileName in f_names:
        if(root not in label_maker.keys()):
            label_maker.update({root : next_free_slot})
            next_free_slot = next_free_slot + 1

        dice_roll = random.randint(1, 100)
        video_path = f"dataset_root/{root}/{fileName}"
        entry = [ video_path , label_maker[root] ] 

        if(dice_roll <= 10):
            test_set.append(entry)
        elif(dice_roll <= 20):
            val_set.append(entry)
        else:
            train_set.append(entry)

# shuffling the entries & adding the header
test_set = random.sample(test_set, len(test_set))
test_set = [['video', 'label']] + test_set

val_set = random.sample(val_set, len(val_set))
val_set = [['video', 'label']] + val_set

train_set = random.sample(train_set, len(train_set))
train_set = [['video', 'label']] + train_set


with open('test.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter= ',')
    writer.writerows(test_set)

with open('val.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter= ',')
    writer.writerows(val_set)

with open('train.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter= ',')
    writer.writerows(train_set)


print(next_free_slot)
            

        

